In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import pandas as pd
import gc

In [98]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [2]:
!ls

'BETO twitter.ipynb'   results		    tweets_mujeres.csv
 README.md	       tweets_hombres.csv


Columnas que se preservan de mis datos: nombre, puesto, usuario, (U)ubicación, fecha d einicio, fecha de término, seguidores, tweets hechos, tweets respuesta, componente sg, edad, comentarios,

Columnas de Twint: id, conversation id, created at, date, timezone, place, tweet, language, hashtags, cashtags, user_id, user id str, username, name, day, hour, link, urls, photos, video, thmbnail, retweet, nlikes, nreplies, nretweets, quote_url, search, near, geo, source, user rt id, user rt, retweet_id, reply_to, retweet date, translate, trans src, trans dest

Columnas con las que me puedo quedar: id, tweet, language, hastags, nlikes, nreplies, nretweets.

In [3]:
mujeres = pd.read_csv("tweets_mujeres.csv",index_col=0)


In [4]:
hombres = pd.read_csv("tweets_hombres.csv",index_col=0)

Hasta aquí tenemos los datos separados en esfera de hombres y esfera de mujeres.

In [5]:
#import torch
import transformers
from transformers import BertForMaskedLM, BertTokenizer
from transformers import AlbertForMaskedLM, AlbertTokenizer
from datasets import Dataset
import datasets

In [6]:
import math

In [7]:
#import torchtext
from collections import Counter
#from torchtext.vocab import Vocab
#from torch import Tensor
import io
import time

Emparejar de manera aleatoria los datasets

In [8]:
import numpy as np

np.random.seed(10)

remove_n = len(hombres) - len(mujeres)
drop_indices = np.random.choice(hombres.index, remove_n, replace=False)
hombres = hombres.drop(drop_indices)

In [9]:
mujeres.columns

Index(['id', 'tweet', 'language', 'hashtags', 'nlikes', 'nreplies',
       'nretweets'],
      dtype='object')

In [10]:
data = {"mujeres_tweet":list(mujeres.tweet),
        "hombres_tweet":list(hombres.tweet)}
raw_dataset = pd.DataFrame(data)
raw_dataset

,mujeres_tweet,hombres_tweet
0,@adams_zulema Muchas gracias @adams_zulema ❤️ ...,@CarolAntonioA Llavas como nueve años de Diput...
1,@adams_zulema buenos días Zulema ❤️ que tenga ...,"@CarolAntonioA Estimado amigo, muchas gracias!..."
2,@adams_zulema que pase buenas noches Zulema ❤️...,Ahora le corresponde a las y los legisladores ...
3,@adams_zulema Que bonita 😍 @adams_zulema ❤️,@CarolAntonioA @LAMelgar El pueblo mexicano es...
4,@adams_zulema @JonBenjamin19 Sin duda un asunt...,@CarolAntonioA te llegaron al precio? Por qué ...
...,...,...
201033,@Nancydelasa @gpplural Descansen en paz 🙏,@lopezobrador_
201034,@Nancydelasa @smontalvod @MariLoliPellon @edua...,@lopezobrador_ Porque detuvieron a Murillo Ka...
201035,@Nancydelasa @senadomexicano @gpplural Excelen...,@lopezobrador_ #SOSCuba Según un ex oficial d...
201036,@Nancydelasa @senadomexicano @gpplural Senador...,@lopezobrador_ @BeatrizGMuller @CarlosLoret @l...


In [11]:
beto = 'dccuchile/bert-base-spanish-wwm-cased'

In [12]:
AL = "CenIA/albert-tiny-spanish"

Convertir de DataFrame a objeto Dataset para Hugginface

In [104]:
ds = Dataset.from_pandas(raw_dataset,preserve_index=True)

In [14]:
ds = ds.rename_column('__index_level_0__','id')

In [105]:
ds

Dataset({
    features: ['mujeres_tweet', 'hombres_tweet', '__index_level_0__'],
    num_rows: 201038
})

In [106]:
ds = ds.train_test_split(shuffle = True, seed = 200, test_size=0.05)

In [107]:
ds

DatasetDict({
    train: Dataset({
        features: ['mujeres_tweet', 'hombres_tweet', '__index_level_0__'],
        num_rows: 190986
    })
    test: Dataset({
        features: ['mujeres_tweet', 'hombres_tweet', '__index_level_0__'],
        num_rows: 10052
    })
})

In [18]:
ds["train"][0]

{'mujeres_tweet': '@Claudiashein Muy bieen',
 'hombres_tweet': '@alfredodelmazo #SalarioRosa Un apoyo muy importante para las mujeres mexiquenses 🌞',
 'id': 77940}

Split datasets - 20% para test.

## Tokenization

In [22]:
tokz = BertTokenizer.from_pretrained(beto, do_lower_case=False,unk_token='[UNK]')

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/364 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [19]:
tokz2 = AlbertTokenizer.from_pretrained(AL, do_lower_case=False)

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

In [24]:
tokz.tokenize("hola mundo; soy Pao.")

['hola', 'mundo', ';', 'soy', 'Pa', '##o', '.']

In [25]:
ds["train"].features

{'mujeres_tweet': Value(dtype='string', id=None),
 'hombres_tweet': Value(dtype='string', id=None),
 'id': Value(dtype='int64', id=None)}

In [20]:
tokz2

PreTrainedTokenizer(name_or_path='CenIA/albert-tiny-spanish', vocab_size=31000, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [21]:
max_input_length = 280
max_target_length = 280
source_lang = "es"
target_lang = "es"
prefix = "translate spanish_to_women to spanish_to_men: "

def preprocess_function(examples):
    inputs = [ex for ex in examples["mujeres_tweet"]]
    targets = [ex for ex in examples["hombres_tweet"]]

    model_inputs = tokz2(inputs,
                        text_pair=targets,
                        padding="max_length", 
                        truncation=True,
                        max_length=max_input_length,
                        #return_tensors = 'pt'
                        )

    #model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
ds["train"]["mujeres_tweet"][:5]

['@Claudiashein Muy bieen',
 '@kenialopezr Wooooo si izo su tarea conocer toda la cdmx q inteligente salió da pena',
 '@rocionahle Buena respuesta al tomandante borolas.',
 '@kenialopezr Todo el país debería tener la seguridad y tranquilidad que se dan los gobiernos panistas, por ejemplo Guanajuato y Tamaulipas!! 😂 Son mediocre, mentirosos, hipócritas y muy malos para gobernar, creen que ya olvidamos lo que pasó con el sin sesos de Fox y el comandante borolas?  https://t.co/9ZHqW4CDBQ',
 '@kenialopezr Vas y chingas tu puta madre,cara de caballo,si vienes por mi alcaldía,probaras un poco del amor del pueblo.']

In [23]:
res = not all(ds["train"]["hombres_tweet"])
 
# printing result
print("Does Dictionary contain None value? " + str(res))

Does Dictionary contain None value? False


In [24]:
en_sentence = ds["train"][100]["mujeres_tweet"]
fr_sentence = ds["train"][100]["hombres_tweet"]

inputs = tokz2(en_sentence, text_pair=fr_sentence)
inputs

{'input_ids': [2, 30924, 1, 105, 30936, 13169, 4704, 38, 30924, 1, 314, 171, 1, 18650, 30924, 1, 30927, 3528, 1797, 11, 6777, 1907, 8858, 41, 2074, 331, 3, 30924, 1, 36, 16733, 66, 6438, 141, 539, 30924, 1, 1335, 30924, 1, 30953, 1377, 8571, 10508, 42, 1439, 11, 30924, 1, 9130, 11, 30481, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
tokenized_dataset = ds.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f8a00720af0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/161 [00:00<?, ?ba/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


  0%|          | 0/41 [00:00<?, ?ba/s]

In [26]:
tokenized_dataset["train"]

Dataset({
    features: ['mujeres_tweet', 'hombres_tweet', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 160830
})

In [27]:
gc.collect()

26

## Transformer (fingers crossed)

In [159]:
from transformers import AutoConfig, AutoModel
from transformers import TrainingArguments,Trainer, DataCollatorForLanguageModeling
from transformers import AutoModelForMaskedLM
from transformers import AlbertForPreTraining
from transformers import AlbertConfig

Descargar la configuracion de BETO

In [29]:
gc.collect()

22

In [30]:
config = AlbertConfig.from_pretrained(AL,low_cpu_mem_usage=True)

Downloading:   0%|          | 0.00/828 [00:00<?, ?B/s]

In [170]:
model2 = AlbertForMaskedLM.from_pretrained(AL)

loading configuration file https://huggingface.co/CenIA/albert-tiny-spanish/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/cf8fc9a69341d4786df7c595b38c52f01aa77e7322ac1c72404199d9e264d925.2036378d93e01dc2b984f4c94384357280e47158e36554e54166bda50cdc7262
Model config AlbertConfig {
  "_name_or_path": "../convert_models/albert_tiny_spanish/converted",
  "architectures": [
    "AlbertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 1248,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 4,
  "num_memory_blocks": 0,
  "pad_t

In [32]:
d_collator = DataCollatorForLanguageModeling(tokenizer=tokz2,return_tensors='pt')

## Training preliminar con Optuna

In [33]:
#import optuna
from transformers import create_optimizer, AdamW, Adafactor

In [54]:
from accelerate import Accelerator

In [177]:
accelerator = Accelerator(mixed_precision='fp16')
model, optimizer = accelerator.prepare(model2,Adafactor)

In [34]:
gc.collect()

2960

In [36]:
import torch
torch.cuda.empty_cache()

In [37]:
#Curso de Jeremy Howard
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [183]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    optim="adafactor",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=1,
    fp16=True,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [184]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokz2,
    data_collator=d_collator,
    compute_metrics=corr_d
)

Using cuda_amp half precision backend


In [185]:
torch.cuda.empty_cache() 

In [186]:
gc.collect()

3137

In [187]:
model2.zero_grad()

In [188]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForMaskedLM.forward` and have been ignored: hombres_tweet, mujeres_tweet, id. If hombres_tweet, mujeres_tweet, id are not expected by `AlbertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 160830
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 628


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `AlbertForMaskedLM.forward` and have been ignored: hombres_tweet, mujeres_tweet, id. If hombres_tweet, mujeres_tweet, id are not expected by `AlbertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40208
  Batch size = 4


RuntimeError: CUDA out of memory. Tried to allocate 15.65 GiB (GPU 0; 79.17 GiB total capacity; 46.22 GiB already allocated; 15.62 GiB free; 62.09 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF